<b>A pulsar is a highly magnetized rotating neutron star or white dwarf that emits a beam of electromagnetic radiation. This radiation can be observed only when the beam of emission is pointing toward Earth (much like the way a lighthouse can be seen only when the light is pointed in the direction of an observer), and is responsible for the pulsed appearance of emission.</b>

<img src='https://upload.wikimedia.org/wikipedia/commons/thumb/a/a6/PIA18848-PSRB1509-58-ChandraXRay-WiseIR-20141023.jpg/223px-PIA18848-PSRB1509-58-ChandraXRay-WiseIR-20141023.jpg'></img>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['pulsar_stars.csv']


<center><font size=4px>A look at the data</font></center>

In [2]:
data=pd.read_csv('../input/pulsar_stars.csv')
data.sample(5)


,Mean of the integrated profile,Standard deviation of the integrated profile,Excess kurtosis of the integrated profile,Skewness of the integrated profile,Mean of the DM-SNR curve,Standard deviation of the DM-SNR curve,Excess kurtosis of the DM-SNR curve,Skewness of the DM-SNR curve,target_class
5181,99.000000,48.547174,0.368495,0.208029,1.531773,15.115768,11.054746,133.638097,0
5128,107.093750,37.854251,0.336808,1.152894,1.090301,11.035449,14.950228,277.888610,0
13387,101.375000,45.359329,0.690028,1.009822,0.722408,9.284064,19.994352,491.366841,0
13814,119.679688,47.904843,0.069294,-0.275621,7.906355,35.213818,4.641162,21.088164,0
10530,125.554688,49.731279,0.064729,0.100816,1.437291,12.104005,12.235932,193.832144,0


<center><font size=4px>Correlation of columns</font></center>

In [3]:
cols=data.columns
for col in cols:
    print('Column:' + str(col))
    print(np.corrcoef(data['target_class'],data[col]))

Column: Mean of the integrated profile
[[ 1.         -0.67318058]
 [-0.67318058  1.        ]]
Column: Standard deviation of the integrated profile
[[ 1.         -0.36370821]
 [-0.36370821  1.        ]]
Column: Excess kurtosis of the integrated profile
[[1.         0.79159138]
 [0.79159138 1.        ]]
Column: Skewness of the integrated profile
[[1.         0.70952802]
 [0.70952802 1.        ]]
Column: Mean of the DM-SNR curve
[[1.         0.40087609]
 [0.40087609 1.        ]]
Column: Standard deviation of the DM-SNR curve
[[1.         0.49153506]
 [0.49153506 1.        ]]
Column: Excess kurtosis of the DM-SNR curve
[[ 1.         -0.39081632]
 [-0.39081632  1.        ]]
Column: Skewness of the DM-SNR curve
[[ 1.        -0.2591167]
 [-0.2591167  1.       ]]
Column:target_class
[[1. 1.]
 [1. 1.]]


<center><font size=4px>Encoding the target class</font></center>

In [4]:
d=pd.get_dummies(data['target_class'])
data=pd.concat([data,d],axis=1)
data.drop('target_class',axis=1,inplace=True)
data.head()

,Mean of the integrated profile,Standard deviation of the integrated profile,Excess kurtosis of the integrated profile,Skewness of the integrated profile,Mean of the DM-SNR curve,Standard deviation of the DM-SNR curve,Excess kurtosis of the DM-SNR curve,Skewness of the DM-SNR curve,0,1
0,140.562500,55.683782,-0.234571,-0.699648,3.199833,19.110426,7.975532,74.242225,1,0
1,102.507812,58.882430,0.465318,-0.515088,1.677258,14.860146,10.576487,127.393580,1,0
2,103.015625,39.341649,0.323328,1.051164,3.121237,21.744669,7.735822,63.171909,1,0
3,136.750000,57.178449,-0.068415,-0.636238,3.642977,20.959280,6.896499,53.593661,1,0
4,88.726562,40.672225,0.600866,1.123492,1.178930,11.468720,14.269573,252.567306,1,0


<center><font size=4px>Making test-set</font></center>

In [5]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(data.drop([0,1],axis=1),data[[0,1]],test_size=0.2)

<center><font size=4px>Neural network construction</font></center>

In [6]:
!pip install keras-metrics

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import adam
import keras_metrics as km
model=Sequential()
for i in range(0,4):
    model.add(Dense(100,input_shape=(8,),activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,input_shape=(8,),activation='relu'))
model.add(Dense(64,input_shape=(8,),activation='relu'))
model.add(Dense(64,input_shape=(8,),activation='relu'))
model.add(Dense(64,input_shape=(8,),activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32,input_shape=(8,),activation='relu'))
model.add(Dense(32,input_shape=(8,),activation='relu'))
model.add(Dense(32,input_shape=(8,),activation='relu'))
model.add(Dense(32,input_shape=(8,),activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2,activation='softmax'))
print('Model summary: ')
model.summary()
print('')
opt = adam(lr=0.001, decay=1e-6)
model.compile(
    loss='categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy',km.precision(),km.recall()])
print('Model fitting...')
model.fit(x_train,y_train, epochs=20, batch_size=32)
score=model.evaluate(x_test,y_test)
print('Accuracy(on Test-data): ' + str(score[1]))


Using TensorFlow backend.


Model summary: 
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 100)               900       
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_4 (Dense)              (None, 100)               10100     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 64)                6464      
_________________________________________________________________
dense_6 (Dense)              (None, 64)                4160 

<center><font size=4px>We have also achieved high precision & recall values. Since the data was heavily skewed, we can use F1 score = (2*P*R)/(P+R) for accurate measurement of our model's performance! <br>Suggestions & feedback always welcomed.</font></center>